In [28]:
import pandas as pd
import json


In [29]:
df = pd.read_csv('dataset/scopusToCSV.csv')

In [30]:
df.loc[0]

id                                              SCOPUS_ID:85186955255
title               Perspectives on label-free microscopy of heter...
aggregation_type                                              Journal
subtype                                                       Article
cited_by_cnt                                                        0
publication_name                         Journal of biomedical optics
publisher                                                         NaN
volume                                                             29
coverDate                                                  2025-12-01
surname                                                          Pham
given_name                                                     Dan L.
affiliation         [{'affilname': 'UW-Madison College of Engineer...
Name: 0, dtype: object

In [31]:
import numpy as np


condition = (df['affiliation'] == "['affilname', 'affiliation-city', 'affiliation-country']")
df.loc[condition, 'affiliation'] = np.NaN

In [32]:
import ast

condition = pd.notna(df['affiliation'])
df.loc[condition, 'affiliation'] = df.loc[condition, 'affiliation'].apply(lambda x: ast.literal_eval(x))

In [33]:
df['full_name1'] = df.agg('{0[given_name]} {0[surname]}'.format, axis=1)
df['full_name1'].unique().size

4885

In [34]:
df['full_name2'] = df['given_name'] + " " +  df['surname']
df['full_name2'].unique().size

4881

In [35]:
df['surname'].isnull().sum()

0

In [36]:
def concat_name(x):
    if pd.isnull(x['given_name']):
        return x['surname']
    return x['given_name'] + " " +  x['surname']

df['full_name'] = df.apply(concat_name, axis=1)
full_name_list = df['full_name'].unique()
print(full_name_list)

['Dan L. Pham' 'Xiaoyun Chen' 'Bolin Li' ... 'Marzhan Nyssanbek'
 'Manish K. Yadav' 'A. Koulogiannis']


In [37]:
df['cited_by_cnt'].max()

12

In [38]:
data_4 = df.groupby('full_name')['cited_by_cnt'].sum().reset_index().sort_values('cited_by_cnt', ascending=False)
data_4 = data_4[data_4['cited_by_cnt'] > 0]
data_4 = pd.concat([data_4, pd.DataFrame({'full_name': 'others','cited_by_cnt': 0}, index=[0])], ignore_index=True)
# data_4

In [39]:
df = df.explode('affiliation')
# df

In [40]:
def split_affiliation_col(x):
    if pd.isna(x):
        return np.nan, np.nan, np.nan
    else:
        return x['affilname'], x['affiliation-city'], x['affiliation-country']


In [41]:
df_split = df.copy()
# df_split[['affilname','affiliation-city','affiliation-country']] = df_split['affiliation'].apply(split_affiliation_col)
# df_split

split_data = df_split['affiliation'].apply(split_affiliation_col).apply(pd.Series)

# # # Rename the columns of the split_data DataFrame
split_data.columns = ['affilname', 'affiliation-city', 'affiliation-country']

# # Assign the split data to the corresponding columns in df_split
df_split[['affilname', 'affiliation-city', 'affiliation-country']] = split_data
# split_data

In [42]:
!pip install pycountry-convert
!pip install pycountry

In [43]:
import pycountry
import pycountry_convert as pc

In [44]:
def get_continent(country_name):
    try:
        country_alpha_2 = pycountry.countries.lookup(country_name).alpha_2
        continent_code = pc.country_alpha2_to_continent_code(country_alpha_2)
        continent_name = pc.convert_continent_code_to_continent_name(continent_code)
        return continent_name
    except LookupError:
        return np.NaN

In [45]:
print(get_continent("turkey"))

nan


In [46]:
df2 = pd.read_csv('dataset/scopusToCSV_FromAjarn.csv')

In [47]:
condition = (df2['affiliation'] == "['affilname', 'affiliation-city', 'affiliation-country']") | \
                (df2['affiliation'] == "['affiliation-city', '@id', 'affilname', '@href', 'affiliation-country']" )
df2.loc[condition, 'affiliation'] = np.NaN
condition = pd.notna(df2['affiliation'])
df2.loc[condition, 'affiliation'] = df2.loc[condition, 'affiliation'].apply(lambda x: ast.literal_eval(x))
# df2

In [48]:
%pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [49]:
df2 = df2.explode('affiliation')

In [50]:
split_data = df2['affiliation'].apply(split_affiliation_col).apply(pd.Series)
split_data.columns = ['affilname', 'affiliation-city', 'affiliation-country']
df2[['affilname', 'affiliation-city', 'affiliation-country']] = split_data
    
df2['region'] = df2['affiliation-country'].apply(get_continent)

In [51]:
all_df = pd.concat([df2, df_split])

In [52]:
unique_rows = all_df.drop_duplicates(subset=['affiliation-city', 'affiliation-country']) 
unique_rows = unique_rows[['affiliation-city', 'affiliation-country']]

In [53]:
%pip install geopandas

Note: you may need to restart the kernel to use updated packages.


In [54]:
from geopy.geocoders import Nominatim

import time


def get_coordinates(idx, x):
    if pd.isna(x).all():
        return np.NaN, np.NaN
    
    country = x['affiliation-country']
    city = x['affiliation-city']

    geolocator = Nominatim(user_agent=f"geolocation-{idx}-test2")

    # Introduce a delay of 1 second
    time.sleep(1)

    if pd.isna(country):
        country = np.NaN
        query = city
    elif pd.isna(city):
        city = np.NaN
        try:
            query = pc.country_name_to_country_alpha2(country, cn_name_format="default")
        except ValueError:
            return np.NaN, np.NaN
    else: 
        query = f"{city} {country}"
    location = geolocator.geocode(query)
   
    if location:
        latitude = location.latitude
        longitude = location.longitude 
        if pd.isna(country):
            pq = f"{city}, "
        elif pd.isna(city):
            pq = f" , {country}"
        else:
            pq = f"{city}, {country}"
            print( f"{pq}, {latitude}, {longitude},")
        return [city, country, latitude, longitude]
    else:
        return city, country, np.NaN, np.NaN

In [55]:
responses = []

In [56]:
# for index, row in unique_rows.iterrows():
#     responses.append(get_coordinates(index, row))
# # unique_rows[['affiliation-city', 'affiliation-country']].apply(get_coordinates, axis=1)

In [57]:
unique_rows.shape

(4226, 2)

In [58]:
unique_rows.to_csv('responses.csv', index=False)

In [64]:
coord_df = pd.read_csv("all_city_country/coordinates/coordinates.csv")
coord_df = coord_df.rename(columns={
                "original_affiliation-city": "affiliation-city",
                "original_affiliation-country": "affiliation-country",
                "lat":"latitude",
                "lon": "longitude"
            })
# coord_df["affiliation-city", "affiliation-country","latitude", "longitude"]
df_test = pd.merge(df2, coord_df[["affiliation-city", "affiliation-country","latitude", "longitude"]], on=["affiliation-city", "affiliation-country"], how='left')

In [74]:
data = df_test.groupby(["latitude", "longitude"])['id'].nunique().reset_index() \
            .rename(columns={'id': 'count'}).sort_values(by='count', ascending=False)
data

,latitude,longitude,count
668,13.752494,100.493509,15580
697,14.020396,100.534043,1747
676,13.819007,100.063992,1411
2941,51.507446,-0.127765,966
1837,37.566679,126.978291,880
...,...,...,...
2447,45.053475,9.694746,1
2446,45.003229,11.083693,1
2445,44.985786,19.630730,1
467,7.293121,80.635036,1


In [67]:
data3 = df_test[["latitude", "longitude",'region', 'id']]

In [68]:
data3

,latitude,longitude,region,id
0,37.427467,-122.170244,North America,SCOPUS_ID:85077976956
1,13.752494,100.493509,Asia,SCOPUS_ID:85077976956
2,13.752494,100.493509,Asia,SCOPUS_ID:85077976956
3,37.427467,-122.170244,North America,SCOPUS_ID:85077976956
4,37.444329,-122.159847,North America,SCOPUS_ID:85077976956
...,...,...,...,...
203227,51.507446,-0.127765,Europe,SCOPUS_ID:85110903700
203228,13.752494,100.493509,Asia,SCOPUS_ID:85106740832
203229,16.430392,102.835590,Asia,SCOPUS_ID:85106740832
203230,13.752494,100.493509,Asia,SCOPUS_ID:85106046890


In [86]:
df_test.groupby(['aggregation_type'])['id'].count().reset_index()
l = df_test['aggregation_type'].unique()
for i in l:
    print(i)
l

Book
Conference Proceeding
Journal
Book Series
Trade Journal


array(['Book', 'Conference Proceeding', 'Journal', 'Book Series',
       'Trade Journal'], dtype=object)